<a href="https://colab.research.google.com/github/NihaarikaAgarwal/Fairness-in-AI-Systems/blob/main/Adult_Income_Prediction_Fairness_Paper_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid')
sns.set_palette('deep')

np.random.seed(7)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from collections import Counter

In [ ]:
df = pd.read_csv('adult.csv')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
2,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
3,40,Private,85019,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,45,?,>50K
4,34,Private,238588,Some-college,10,Never-married,Other-service,Own-child,Black,Female,0,0,35,United-States,<=50K


# Data cleaning


1.   Check if there are any null values
2.   Check if there are any special characters in the Dtype object columns
3.   Drop columns with very few values, scale numerical values where required
3.   One-hot encoding categorical features



In [ ]:
df.isnull().sum().sort_index()

age                0
capital-gain       0
capital-loss       0
education          0
educational-num    0
fnlwgt             0
gender             0
hours-per-week     0
income             0
marital-status     0
native-country     0
occupation         0
race               0
relationship       0
workclass          0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14715 entries, 0 to 14714
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              14715 non-null  int64 
 1   workclass        14715 non-null  object
 2   fnlwgt           14715 non-null  int64 
 3   education        14715 non-null  object
 4   educational-num  14715 non-null  int64 
 5   marital-status   14715 non-null  object
 6   occupation       14715 non-null  object
 7   relationship     14715 non-null  object
 8   race             14715 non-null  object
 9   gender           14715 non-null  object
 10  capital-gain     14715 non-null  int64 
 11  capital-loss     14715 non-null  int64 
 12  hours-per-week   14715 non-null  int64 
 13  native-country   14715 non-null  object
 14  income           14715 non-null  object
dtypes: int64(6), object(9)
memory usage: 1.7+ MB


In [ ]:
categorical_features = df.select_dtypes(include =['object']).axes[1]
for col in categorical_features:
  print(col, ": \n", df[col].unique(), end="\n\n")

workclass : 
 ['Private' '?' 'State-gov' 'Federal-gov' 'Local-gov' 'Self-emp-not-inc'
 'Self-emp-inc' 'Never-worked' 'Without-pay']

education : 
 ['11th' 'Some-college' 'HS-grad' 'Doctorate' '7th-8th' 'Masters'
 'Bachelors' 'Assoc-acdm' '1st-4th' 'Assoc-voc' '9th' '5th-6th' '10th'
 'Prof-school' 'Preschool' '12th']

marital-status : 
 ['Never-married' 'Married-civ-spouse' 'Separated' 'Divorced'
 'Married-spouse-absent' 'Widowed' 'Married-AF-spouse']

occupation : 
 ['Machine-op-inspct' '?' 'Prof-specialty' 'Other-service'
 'Exec-managerial' 'Priv-house-serv' 'Handlers-cleaners' 'Craft-repair'
 'Adm-clerical' 'Transport-moving' 'Sales' 'Farming-fishing'
 'Tech-support' 'Protective-serv' 'Armed-Forces']

relationship : 
 ['Own-child' 'Husband' 'Unmarried' 'Not-in-family' 'Wife' 'Other-relative']

race : 
 ['Black' 'Asian-Pac-Islander' 'Other' 'Amer-Indian-Eskimo' 'White']

gender : 
 ['Male' 'Female']

native-country : 
 ['United-States' '?' 'Dominican-Republic' 'Germany' 'Philippines'


In [ ]:
df['workclass'] = df['workclass'].replace({'?':'Other'})
df['occupation'] = df['occupation'].replace({'?':'Other'})
df['native-country'] = df['native-country'].replace({'?':'Other'})
df['income']=df['income'].map({'<=50K': 0, '>50K': 1})

In [ ]:
df.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income
count,14715.000000,1.471500e+04,14715.000000,14715.000000,14715.000000,14715.000000,14715.000000
mean,38.218009,1.954787e+05,9.952633,981.486306,80.453347,39.915392,0.201631
std,13.388829,1.120910e+05,2.574854,7275.352726,385.898048,11.902639,0.401232
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.194435e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,36.000000,1.805320e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,47.000000,2.471085e+05,12.000000,0.000000,0.000000,40.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [ ]:
# 75% values are 0
df.drop(['capital-gain', 'capital-loss', 'fnlwgt'], axis=1 ,inplace = True)
# df['fnlwgt'] = df['fnlwgt'].apply(lambda x: np.log1p(x))

In [ ]:
df['educational-num'].unique()

array([ 7, 10,  9, 16,  4, 14, 13, 12,  2, 11,  5,  3,  6, 15,  1,  8])

In [ ]:
# similar to education column
df.drop('educational-num', axis=1 ,inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14715 entries, 0 to 14714
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             14715 non-null  int64 
 1   workclass       14715 non-null  object
 2   education       14715 non-null  object
 3   marital-status  14715 non-null  object
 4   occupation      14715 non-null  object
 5   relationship    14715 non-null  object
 6   race            14715 non-null  object
 7   gender          14715 non-null  object
 8   hours-per-week  14715 non-null  int64 
 9   native-country  14715 non-null  object
 10  income          14715 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 1.2+ MB


In [ ]:
categorical_features = df.select_dtypes(include = ['object']).axes[1]
numerical_features = list(set(df.columns) - set(categorical_features))

for col in categorical_features:
    df = pd.concat([df , pd.get_dummies(df[col], prefix = col , prefix_sep = ':')] , axis = 1)
df.drop(categorical_features , axis = 1 , inplace = True)
categorical_features = list(set(df.columns) - set(numerical_features))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14715 entries, 0 to 14714
Columns: 104 entries, age to native-country:Yugoslavia
dtypes: int64(3), uint8(101)
memory usage: 1.8 MB


# Train-test split

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=69, shuffle=True)

# Bias detection and mitigation



In [ ]:
protected_feature = 'gender'
privilege_class = 'Male'
output_feature = 'income'

In [ ]:
def calculate_bias_metrics(df_privilege, df_unprivilege, protected_feature, privilege_class, output_feature):
  total_privilege, total_unprivilege = df_privilege.shape[0], df_unprivilege.shape[0]

  fav_outcome_privilege, fav_outcome_unprivilege = df_privilege.loc[df_privilege[output_feature] == 1].shape[0], df_unprivilege.loc[df_unprivilege[output_feature] == 1].shape[0]

  prob_fav_outcome_privilege = fav_outcome_privilege/total_privilege
  prob_fav_outcome_unprivilege = fav_outcome_unprivilege/total_unprivilege

  print("Probability of fav outcome for privelege class: ", prob_fav_outcome_privilege)

  print("Probability of fav outcome for unprivelege class: ", prob_fav_outcome_unprivilege)

  print("Statistical parity difference: ", prob_fav_outcome_unprivilege - prob_fav_outcome_privilege)

  print("Disparate impact: ", prob_fav_outcome_unprivilege / prob_fav_outcome_privilege)

  print("Euclidean distance: ", np.sqrt(np.mean([(a-b)*(a-b) for a, b in zip(df_privilege.mean(), df_unprivilege.mean())])))

  print("Manhattan distance: ", np.mean([np.abs(a-b) for a, b in zip(df_privilege.mean(), df_unprivilege.mean())]))


In [ ]:
df_train_privilege = df_train.loc[df_train[protected_feature + ':' + privilege_class] == 1]
df_train_unprivilege = df_train.loc[df_train[protected_feature + ':' + privilege_class] == 0]

calculate_bias_metrics(df_train_privilege.copy(deep=True), df_train_unprivilege.copy(deep=True), protected_feature, privilege_class, output_feature)

Probability of fav outcome for privelege class:  0.26364166909833675
Probability of fav outcome for unprivelege class:  0.09110473457675754
Statistical parity difference:  -0.1725369345215792
Disparate impact:  0.34556272871560384
Euclidean distance:  0.5578921465794333
Manhattan distance:  0.11825568281644469


In [ ]:
X_train = df_train.drop(output_feature, axis=1)
y_train = df_train[output_feature]

X_test = df_test.drop(output_feature, axis=1)
y_test = df_test[output_feature]

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

X_test = scaler.transform(X_test)
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test , y_pred))

0.8350095134547432


In [ ]:
df_test_previlege = df_test.loc[df_test[protected_feature + ':' + privilege_class] == 1]
df_test_unprevilege = df_test.loc[df_test[protected_feature + ':' + privilege_class] == 0]

X_test_previlege = df_test_previlege.drop(output_feature, axis=1)
y_test_previlege = df_test_previlege[output_feature]
X_test_unprevilege = df_test_unprevilege.drop(output_feature, axis=1)
y_test_unprevilege = df_test_unprevilege[output_feature]

In [ ]:
X_test_previlege = scaler.transform(X_test_previlege)
y_pred_previlege = classifier.predict(X_test_previlege)
print(accuracy_score(y_test_previlege , y_pred_previlege))

0.7897503285151117


In [ ]:
X_test_unprevilege = scaler.transform(X_test_unprevilege)
y_pred_unprevilege = classifier.predict(X_test_unprevilege)
print(accuracy_score(y_test_unprevilege , y_pred_unprevilege))

0.9090257879656161


In [ ]:
def bias_mitigation(dataset, categorical_features, output_feature, ratio):
  dataset_X = dataset.drop(output_feature, axis=1)
  dataset_Y = dataset[output_feature]

  print(dataset_Y.value_counts(normalize=True))

  cat_col_indices = [dataset_X.columns.get_loc(col) for col in categorical_features]
  cat_col_indices.sort()

  # pipeline for oversampling and undersampling
  over_sampling = SMOTENC(categorical_features=cat_col_indices, sampling_strategy = ratio - 0.1)
  under_sampling = RandomUnderSampler(sampling_strategy = ratio)
  steps = [('o', over_sampling), ('u', under_sampling)]
  # steps = [('u', under_sampling)]
  pipeline = Pipeline(steps=steps)

  X, y = pipeline.fit_resample(dataset_X, dataset_Y)

  print(pd.DataFrame(y).value_counts(normalize=True))

  X = pd.DataFrame(X)
  y = pd.DataFrame(y, columns=[output_feature])

  return pd.concat([X, y], axis = 1)

In [ ]:
df_train_privilege_resampled = bias_mitigation(df_train_privilege.copy(deep=True), categorical_features, output_feature, 0.8)
df_train_unprivilege_resampled = bias_mitigation(df_train_unprivilege.copy(deep=True), categorical_features, output_feature, 0.8)

0    0.696692
1    0.303308
Name: income, dtype: float64


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

0    0.555547
1    0.444453
dtype: float64
0    0.891071
1    0.108929
Name: income, dtype: float64
0    0.555542
1    0.444458
dtype: float64


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
calculate_bias_metrics(df_train_privilege_resampled.copy(deep=True), df_train_unprivilege_resampled.copy(deep=True), protected_feature, privilege_class, output_feature)

Probability of fav outcome for privelege class:  0.4444527067221892
Probability of fav outcome for unprivelege class:  0.4444575124963246
Statistical parity difference:  4.80577413541905e-06
Disparate impact:  1.0000108127907936
Euclidean distance:  0.5436514789320455
Manhattan distance:  0.11167270252582885


In [ ]:
df_train_resampled = pd.concat([df_train_privilege_resampled, df_train_unprivilege_resampled], axis = 0)

In [ ]:
df_train_resampled[output_feature].value_counts(normalize=True)

0    0.555545
1    0.444455
Name: income, dtype: float64

# Model fitting and prediction

In [ ]:
X_train = df_train_resampled.drop(output_feature, axis=1)
y_train = df_train_resampled[output_feature]
# X_train = df_train.drop(output_feature, axis=1)
# y_train = df_train[output_feature]

X_test = df_test.drop(output_feature, axis=1)
y_test = df_test[output_feature]

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

X_test = scaler.transform(X_test)
y_pred = classifier.predict(X_test)

# Prediction analysis and visualisations

In [ ]:
print("Confusion matrix\n", classification_report(y_test, y_pred))
print("Area under curve score: ", roc_auc_score(y_test, y_pred))

Confusion matrix
               precision    recall  f1-score   support

           0       0.88      0.86      0.87      9279
           1       0.58      0.63      0.60      2932

    accuracy                           0.80     12211
   macro avg       0.73      0.74      0.74     12211
weighted avg       0.81      0.80      0.80     12211

Area under curve score:  0.7434576631325969


In [ ]:
df_test_previlege = df_test.loc[df_test[protected_feature + ':' + privilege_class] == 1]
df_test_unprevilege = df_test.loc[df_test[protected_feature + ':' + privilege_class] == 0]

X_test_previlege = df_test_previlege.drop(output_feature, axis=1)
y_test_previlege = df_test_previlege[output_feature]
X_test_unprevilege = df_test_unprevilege.drop(output_feature, axis=1)
y_test_unprevilege = df_test_unprevilege[output_feature]

In [ ]:
X_test_previlege = scaler.transform(X_test_previlege)
y_pred_previlege = classifier.predict(X_test_previlege)
print("Confusion matrix\n", classification_report(y_test_previlege, y_pred_previlege))
print("Area under curve score: ", roc_auc_score(y_test_previlege, y_pred_previlege))

Confusion matrix
               precision    recall  f1-score   support

           0       0.84      0.81      0.83      5654
           1       0.60      0.64      0.62      2483

    accuracy                           0.76      8137
   macro avg       0.72      0.73      0.72      8137
weighted avg       0.77      0.76      0.76      8137

Area under curve score:  0.7284178683174345


In [ ]:
X_test_unprevilege = scaler.transform(X_test_unprevilege)
y_pred_unprevilege = classifier.predict(X_test_unprevilege)
print("Confusion matrix\n", classification_report(y_test_unprevilege, y_pred_unprevilege))
print("Area under curve score: ", roc_auc_score(y_test_unprevilege, y_pred_unprevilege))

Confusion matrix
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      3625
           1       0.47      0.57      0.51       449

    accuracy                           0.88      4074
   macro avg       0.71      0.75      0.72      4074
weighted avg       0.89      0.88      0.89      4074

Area under curve score:  0.7456398126103986
